### 불법주정차량 시간대와 종류 자동식별

<br>

[종합 민원 이미지 AI데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=492)
<br>[불법주정차량 시간대와 종류 자동식별](https://aifactory.space/task/2665/overview)

<br>

In [2]:
import numpy as np
import sys
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import pandas as pd

2023-11-13 06:57:50.425752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 06:57:50.425785: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 06:57:50.425808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 06:57:50.433089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 06:57:51.152718: W tensorflow/c

In [5]:
img_size = (64, 64)

x_train_path = 'data/data/x_train.npy'
y_train_path = 'data/data/y_train.npy'
model_save_path = 'output'

x_train = np.load(x_train_path)
y_train = np.load(y_train_path)
y_train = to_categorical(y_train)  # 원-핫 인코딩

"""
# 모델 구축
model = keras.Sequential([
    layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(4, activation="softmax")  # 4개의 클래스에 대응하는 출력 뉴런
])
"""
model = keras.models.Sequential([
    layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
    keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    
    keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    keras.layers.Dropout(0.25),

    keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(128, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer

    keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    keras.layers.Dropout(0.25),

    keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer
    keras.layers.Conv2D(256, 3, activation='relu', padding='SAME'), #cnn layer
    keras.layers.BatchNormalization(), #batch norm layer

    keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
    keras.layers.Dropout(0.25),

    keras.layers.Flatten(),
    
    keras.layers.Dense(512, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(4, activation = 'sigmoid') # ouput layer
    ])

# Optimizer 설정
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=32)

model.save(model_save_path)

Epoch 1/20
118/118 [==============================] - 1s 5ms/step - loss: 110.1942 - accuracy: 0.2598
Epoch 2/20
118/118 [==============================] - 1s 5ms/step - loss: 1.3417 - accuracy: 0.3493
Epoch 3/20
118/118 [==============================] - 1s 5ms/step - loss: 1.1425 - accuracy: 0.4425
Epoch 4/20
118/118 [==============================] - 1s 5ms/step - loss: 0.9623 - accuracy: 0.5450
Epoch 5/20
118/118 [==============================] - 1s 5ms/step - loss: 0.8463 - accuracy: 0.6181
Epoch 6/20
118/118 [==============================] - 1s 5ms/step - loss: 0.7903 - accuracy: 0.6457
Epoch 7/20
118/118 [==============================] - 1s 6ms/step - loss: 0.7323 - accuracy: 0.6778
Epoch 8/20
118/118 [==============================] - 1s 5ms/step - loss: 0.6306 - accuracy: 0.7211
Epoch 9/20
118/118 [==============================] - 1s 5ms/step - loss: 0.5977 - accuracy: 0.7461
Epoch 10/20
118/118 [==============================] - 1s 5ms/step - loss: 0.5920 - accuracy: 0.75

INFO:tensorflow:Assets written to: output/assets


In [6]:
import numpy as np
import sys
from tensorflow.keras.models import load_model
import pandas as pd

In [7]:
model_path = 'output'
x_test_path = 'data/data/x_test.npy'
y_pred_save_path = 'y_pred.npy'

model = load_model(model_path)
x_test = np.load(x_test_path)

y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)

np.save(y_pred_save_path, y_pred_labels)

52/52 [==============================] - 0s 3ms/step
